In [1]:
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

In [2]:
from qdrant_client import QdrantClient, models

/Users/rajwantkaur/.pyenv/versions/3.11.3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
client = QdrantClient("http://localhost:6333")

### Q1. Embedding the query

In [4]:
from fastembed import TextEmbedding

In [5]:
query = 'I just discovered the course. Can I join now?'

In [6]:
model_embed = TextEmbedding(model_name = "jinaai/jina-embeddings-v2-small-en")

In [7]:
query_embedding = next(model_embed.embed([query]))

In [8]:
query_embedding.shape

(512,)

In [9]:
print(f"Minimum value in the embedded array is: {query_embedding.min()}")

Minimum value in the embedded array is: -0.11726373885183883


In [10]:
query_embedding.dot(query_embedding)

np.float64(1.0000000000000002)

In [11]:
#Cosine Similarity
import numpy as np
np.linalg.norm(query_embedding)

np.float64(1.0)

### Q2. Cosine similarity with another vector

In [12]:
doc = 'Can I still join the course after the start date?'

In [13]:
doc_embed = next(model_embed.embed([doc]))

In [14]:
doc_embed.dot(query_embedding)

np.float64(0.9008528895674548)

### Q3. Ranking by cosine

In [15]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [16]:
text_embed = list(model_embed.embed([doc['text'] for doc in documents]))

In [17]:
V = np.array(text_embed)

In [18]:
V

array([[-0.0249525 , -0.0396454 , -0.00437672, ...,  0.06779866,
         0.0377203 , -0.00470724],
       [-0.0594709 , -0.08523984,  0.01292091, ...,  0.09599709,
         0.05420843, -0.00029461],
       [-0.06846454, -0.04079098,  0.04998121, ...,  0.06578071,
         0.02872077, -0.01115215],
       [-0.04640506, -0.02555229,  0.02241224, ...,  0.03410312,
         0.06813592, -0.0064033 ],
       [-0.05394912, -0.04693814,  0.00794725, ...,  0.05402997,
         0.03033385, -0.01254518]], shape=(5, 512))

In [19]:
#Cosine similarities
cosine_sim = V.dot(query_embedding)

In [20]:
max_index = np.argmax(cosine_sim)
max_index

np.int64(1)

### Q4. Ranking by cosine, version two

In [21]:

full_text = [doc['question'] + ' ' + doc['text'] for doc in documents]
    
full_text

["Course - Can I still join the course after the start date? Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'Course - Can I follow the course after it finishes? Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 "Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin t

In [22]:
full_text_embed = list(model_embed.embed(full_text))

In [23]:
ft_V = np.array(full_text_embed)

In [24]:
ft_cos_sim = ft_V.dot(query_embedding)

In [25]:
np.argmax(ft_cos_sim)

np.int64(0)

### Q5. Selecting the embedding model

In [ ]:
modelslist = TextEmbedding.list_supported_models()
modelslist


In [28]:
min_dim_model = min(modelslist, key=lambda m: m['dim'])
print(f"The smallest model is {min_dim_model['model']} with dimenssions as {min_dim_model['dim']}.")

The smallest model is BAAI/bge-small-en with dimenssions as 384.


### Q6. Indexing with qdrant 

In [29]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [30]:
EMBEDDING_DIMENSIONALITY = 384
model_handle = "BAAI/bge-small-en"

In [31]:
collection_name = "qdrant-baai"

In [32]:
model_embed2 = TextEmbedding(model_name = model_handle)

In [ ]:
documents

In [34]:
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [35]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [36]:
model_handle

'BAAI/bge-small-en'

In [37]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [38]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [39]:
def search(query, limit=3):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [42]:
result = search(query)

In [49]:
result.points[0].score

0.8703172

In [46]:
print(f"The question in the query is: {query} ")
print(f"Top Retrieved Answer: {result.points[0].payload['text']}")

The question in the query is: I just discovered the course. Can I join now? 
Top Retrieved Answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.
